In [2]:

!pip install -q torch transformers accelerate peft trl bitsandbytes scipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 14.6 MB/s eta 0:00:00


In [3]:
!pip install -q -U transformers peft accelerate bitsandbytes trl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 50.3 MB/s eta 0:00:00


In [4]:
!pip install bitsandbytes==0.41.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.3 MB/s eta 0:00:00
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.48.2
    Uninstalling bitsandbytes-0.48.2:
      Successfully uninstalled bitsandbytes-0.48.2


In [5]:
!pip install -U "trl>=0.8.6" "transformers>=4.44" "peft>=0.12" "bitsandbytes>=0.43" "accelerate>=1.0"

  Using cached bitsandbytes-0.48.2-py3-none-manylinux_2_24_x86_64.whl.metadata (10 kB)
Using cached bitsandbytes-0.48.2-py3-none-manylinux_2_24_x86_64.whl (59.4 MB)
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.41.0
    Uninstalling bitsandbytes-0.41.0:
      Successfully uninstalled bitsandbytes-0.41.0


In [6]:

!pip uninstall -y bitsandbytes accelerate peft transformers trl
!pip install bitsandbytes
!pip install --upgrade accelerate peft trl transformers

Found existing installation: bitsandbytes 0.48.2
Uninstalling bitsandbytes-0.48.2:
  Successfully uninstalled bitsandbytes-0.48.2
Found existing installation: accelerate 1.12.0
Uninstalling accelerate-1.12.0:
  Successfully uninstalled accelerate-1.12.0
Found existing installation: peft 0.18.0
Uninstalling peft-0.18.0:
  Successfully uninstalled peft-0.18.0
Found existing installation: transformers 4.57.3
Uninstalling transformers-4.57.3:
  Successfully uninstalled transformers-4.57.3
Found existing installation: trl 0.25.1
Uninstalling trl-0.25.1:
  Successfully uninstalled trl-0.25.1
  Using cached bitsandbytes-0.48.2-py3-none-manylinux_2_24_x86_64.whl.metadata (10 kB)
Using cached bitsandbytes-0.48.2-py3-none-manylinux_2_24_x86_64.whl (59.4 MB)
  Using cached trl-0.25.1-py3-none-any.whl.metadata (11 kB)
  Using cached transformers-4.57.3-py3-none-any.whl.metadata (43 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:

!pip install -q -U trl transformers peft accelerate bitsandbytes datasets


In [2]:
%%capture
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes


In [5]:
import torch
from unsloth import FastLanguageModel
from datasets import load_dataset
from trl import SFTTrainer
from transformers import TrainingArguments
import os
os.environ["WANDB_DISABLED"] = "true"

max_seq_length = 2048
dtype = None
load_in_4bit = True

print("Incarcare model...")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)


model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # 0 e optimizat pentru Unsloth
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)


alpaca_prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{}<|eot_id|><|start_header_id|>user<|end_header_id|>

{}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{}<|eot_id|>"""

EOS_TOKEN = tokenizer.eos_token

def formatting_prompts_func(examples):
    sys_prompts = []
    users = []
    assistants = []

    for conversation in examples["messages"]:
        sys = ""
        usr = ""
        ast = ""

        for msg in conversation:
            if msg["role"] == "system": sys = msg["content"]
            if msg["role"] == "user": usr = msg["content"]
            if msg["role"] == "assistant": ast = msg["content"]

        sys_prompts.append(sys)
        users.append(usr)
        assistants.append(ast)

    texts = []
    for sys, usr, ast in zip(sys_prompts, users, assistants):
        text = alpaca_prompt.format(sys, usr, ast) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts }


dataset = load_dataset("json", data_files="/content/drive/MyDrive/date_verificate/*.json", split="train")
dataset = dataset.map(formatting_prompts_func, batched = True)

print(f"Date incarcate: {len(dataset)} conversatii.")

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 100,
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",
    ),
)

print("Incepem antrenarea...")
trainer_stats = trainer.train()

print("Salvam modelul...")
model.save_pretrained("llama3_stoma_model")
tokenizer.save_pretrained("llama3_stoma_model")
print("Gata! Modelul este salvat in folderul 'llama3_stoma_model'.")

Incarcare model...
==((====))==  Unsloth 2025.11.4: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/289 [00:00<?, ? examples/s]

Date incarcate: 289 conversatii.


Unsloth: Tokenizing ["text"] (num_proc=6):   0%|          | 0/289 [00:00<?, ? examples/s]

...

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 289 | Num Epochs = 3 | Total steps = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040 of 8,072,204,288 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.864500
2,2.829800
3,2.776800
4,2.528100
5,2.300300
6,2.146900
7,1.944100
8,1.729000
9,1.559600
10,1.343200


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Salvam modelul...
Gata! Modelul este salvat in folderul 'llama3_stoma_model'.


In [4]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [9]:
import os
import shutil
from google.colab import drive


if not os.path.exists('/content/drive'):
    drive.mount('/content/drive')

sursa = "/content/llama3_stoma_model"

destinatie = "/content/drive/MyDrive/llama3_stoma_model"

if os.path.exists(sursa):
    print(f"✅ Am găsit modelul în: {sursa}")
    print(f"⏳ Încep copierea în Google Drive... (poate dura 1-2 minute)")

    !cp -r "{sursa}" "{destinatie}"


    if os.path.exists(destinatie):
        print(f"🎉 GATA! Folderul a fost copiat cu succes aici:")
        print(f"👉 {destinatie}")
        print("Acum poți folosi această cale în scriptul de chat (inference).")
    else:
        print("❌ Ceva nu a mers la copiere. Verifică spațiul din Drive.")

else:
    print(f"❌ EROARE: Nu găsesc folderul '{sursa}'.")
    print("Sigur ai rulat comanda: trainer.model.save_pretrained(...) ?")

✅ Am găsit modelul în: /content/llama3_stoma_model
⏳ Încep copierea în Google Drive... (poate dura 1-2 minute)
🎉 GATA! Folderul a fost copiat cu succes aici:
👉 /content/drive/MyDrive/llama3_stoma_model
Acum poți folosi această cale în scriptul de chat (inference).


In [1]:

!pip install flask flask-cors pyngrok transformers accelerate torch bitsandbytes peft

import torch
import os
from flask import Flask, request, jsonify
from flask_cors import CORS
from pyngrok import ngrok
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
from google.colab import drive

drive.mount('/content/drive')
ADAPTER_DIR = "/content/drive/MyDrive/llama3_stoma_model"

BASE_ID = "unsloth/llama-3-8b-Instruct-bnb-4bit"

print(f"🔄 Se încarcă modelul de bază: {BASE_ID} ...")


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

try:
    base_model = AutoModelForCausalLM.from_pretrained(
        BASE_ID,
        quantization_config=bnb_config,
        device_map="auto",
        trust_remote_code=True
    )
    tokenizer = AutoTokenizer.from_pretrained(BASE_ID, trust_remote_code=True)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token

    print(f"🔄 Se aplică adapterul tău din: {ADAPTER_DIR} ...")
    model = PeftModel.from_pretrained(base_model, ADAPTER_DIR)
    model.eval()

    print("✅ SUCCES: Modelul Llama 3 Fine-Tuned a fost încărcat!")

except Exception as e:
    print(f"\n❌ EROARE CRITICĂ: {e}")
    print("Verifică următoarele:")
    print("1. Calea ADAPTER_DIR este corectă? (Ai fișierul adapter_config.json acolo?)")
    print("2. Ai memorie GPU liberă? (Factory Reset Runtime dacă e plină)")


app = Flask(__name__)
CORS(app)

NGROK_AUTH_TOKEN = "367YlYgVRZnk67B0VrocFGjZfXA_24WscVzsQzmD622DQ6fhn"
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

@app.route('/generate', methods=['POST'])
def generate_endpoint():
    data = request.json
    messages = data.get("messages", [])

    try:
        prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

        # Generare
        with torch.no_grad():
            output = model.generate(
                **inputs,
                max_new_tokens=200,
                temperature=0.2,
                do_sample=True,
                top_p=0.9,
                repetition_penalty=1.1,
                eos_token_id=tokenizer.eos_token_id,
                pad_token_id=tokenizer.pad_token_id
            )

        prompt_length = inputs["input_ids"].shape[-1]
        generated_tokens = output[0, prompt_length:]
        reply = tokenizer.decode(generated_tokens, skip_special_tokens=True).strip()

        return jsonify({"generated_text": reply})

    except Exception as e:
        return jsonify({"error": str(e)}), 500

try:
    ngrok.kill()

    public_url = ngrok.connect(5000).public_url
    print(f"\n🚀 SERVER PORNIT! Link-ul tău este: {public_url}")
    app.run(port=5000)
except Exception as e:
    print(f"Eroare la pornire server: {e}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 13.0 MB/s eta 0:00:00
Mounted at /content/drive
🔄 Se încarcă modelul de bază: unsloth/llama-3-8b-Instruct-bnb-4bit ...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/quantizers/auto.py:239: UserWarning: You passed `quantization_config` or equivalent parameters to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` from the model will be used.
  warnings.warn(warning_msg)


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/345 [00:00<?, ?B/s]

🔄 Se aplică adapterul tău din: /content/drive/MyDrive/llama3_stoma_model ...
✅ SUCCES: Modelul Llama 3 Fine-Tuned a fost încărcat!

🚀 SERVER PORNIT! Link-ul tău este: https://sharon-preperusal-preobediently.ngrok-free.dev
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [29/Nov/2025 20:54:48] "POST /generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Nov/2025 20:56:16] "POST /generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Nov/2025 20:57:25] "POST /generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Nov/2025 20:58:11] "POST /generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Nov/2025 20:58:41] "POST /generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Nov/2025 20:59:53] "POST /generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Nov/2025 21:01:02] "POST /generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Nov/2025 21:02:38] "POST /generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Nov/2025 21:04:16] "POST /generate HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [29/Nov/2025 21:04:38